In [124]:
from lib import read_arr

from enum import Enum

import numpy as np
from tqdm import tqdm

In [119]:
arr = read_arr("16_input.txt")

In [120]:
Dir = Enum("Dir", ["Up", "Right", "Down", "Left"])

steps = {
    Dir.Right: np.array([0, 1]),
    Dir.Left: np.array([0, -1]),
    Dir.Up: np.array([-1, 0]),
    Dir.Down: np.array([1, 0])
}


In [121]:
def n_energised(start_dir, start_idx) -> int:
    visited = {d: np.zeros(arr.shape, dtype=bool) for d in Dir}

    def run_beam(idx, direct):
        idx = idx.copy()

        while True:
            # Take a step
            idx += steps[direct]
            if any(i < 0 for i in idx) or idx[0] >= arr.shape[0] or idx[1] >= arr.shape[1]:
                # Out of bounds
                break
            if visited[direct][tuple(idx)]:
                # Alrady visited
                break
            
            # Mark as visited
            visited[direct][tuple(idx)] = 1
            
            
            symbol = arr[tuple(idx)]
            if symbol == "|" and direct in [Dir.Left, Dir.Right]:
                run_beam(idx, Dir.Up)
                run_beam(idx, Dir.Down)
                break
            elif symbol == "-" and direct in [Dir.Up, Dir.Down]:
                run_beam(idx, Dir.Right)
                run_beam(idx, Dir.Left)
                break
            elif symbol == "/":
                match direct:
                    case Dir.Up:
                        run_beam(idx, Dir.Right)
                    case Dir.Down:
                        run_beam(idx, Dir.Left)
                    case Dir.Left:
                        run_beam(idx, Dir.Down)
                    case Dir.Right:
                        run_beam(idx, Dir.Up)
                break
            elif symbol == "\\":
                match direct:
                    case Dir.Up:
                        run_beam(idx, Dir.Left)
                    case Dir.Down:
                        run_beam(idx, Dir.Right)
                    case Dir.Left:
                        run_beam(idx, Dir.Up)
                    case Dir.Right:
                        run_beam(idx, Dir.Down)
                break
                            


    visited[start_dir][tuple(start_idx)] = 1
    run_beam(start_idx, start_dir)


    all_visited = list(visited.values())[0]
    for val in visited.values():
        all_visited = all_visited | val

    return np.sum(all_visited)


In [122]:
arr.shape

(110, 110)

In [126]:
max_energ = 0
# Top row
for i in tqdm(range(arr.shape[0])):
    n = n_energised(Dir.Right, np.array([i, 0]))
    max_energ = max(max_energ, n)

# Bottom row
for i in tqdm(range(arr.shape[0])):
    n = n_energised(Dir.Left, np.array([i, arr.shape[1] - 1]))
    max_energ = max(max_energ, n)

# Left row
for i in tqdm(range(arr.shape[1])):
    n = n_energised(Dir.Down, np.array([0, i]))
    max_energ = max(max_energ, n)

# Right row
for i in tqdm(range(arr.shape[1])):
    n = n_energised(Dir.Up, np.array([arr.shape[0] - 1, i]))
    max_energ = max(max_energ, n)

max_energ

100%|██████████| 110/110 [00:02<00:00, 48.31it/s]


8183